# Sequence to Sequence Learning with Neural Networks

This notebook implements the model in:  
Sutskever, I., Vinyals, O., and Le, Q. V. 2014. Sequence to sequence learning with neural networks. In Advances in neural information processing systems, 3104-3112. [arXiv:1409.3215](https://arxiv.org/abs/1409.3215). 

This model is based on an *Encoder-Decoder* framework, in which the encoder and the decoder are both RNNs.  
This model encodes all the information of the source sequence into a fixed-length context vector $z$, and then generates the target sequence from $z$.  

![Seq2Seq with NN](fig/seq2seq-with-NN.png)

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
# This function also invokes `torch.cuda.manual_seed_all(SEED)`.
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preparing Data
[Sutskever et al. (2014)](https://arxiv.org/abs/1409.3215) find it beneficial to reverse the order of the source sequence, which they believe "introduces many short term dependencies in the data that make the optimization problem much easier". 

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de_rev(text):
    """
    Tokenize German text and reverse it. 
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenize English text.
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [3]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de_rev, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True)

In [4]:
from torchtext.datasets import Multi30k

train_data, valid_data, test_data = Multi30k.splits(exts=['.de', '.en'], 
                                                    # fields=[SRC, TRG], 
                                                    fields=[('src', SRC), ('trg', TRG)], 
                                                    root='data/')

In [5]:
print(train_data[0].src)
print(train_data[0].trg)

['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [6]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

len(SRC.vocab), len(TRG.vocab)

(7855, 5893)

In [7]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [8]:
for batch in train_iterator:
    batch_src, batch_src_lens = batch.src
    batch_trg, batch_trg_lens = batch.trg
    break
print(batch_src)
print(batch_src_lens)
print(batch_trg)
print(batch_trg_lens)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,    4,  ...,    4,    4,    4],
        [ 293,  913, 3887,  ...,   68, 5563,    0],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
tensor([14, 17, 12, 11, 17, 21, 12, 16, 14, 11, 23, 23,  8, 11,  9, 14, 19, 20,
        12, 16,  9, 11, 13, 20, 21, 29, 13, 22, 14, 16, 10,  9, 15, 12, 17, 10,
        14, 22, 17, 20, 23, 23, 12, 17, 15, 19, 17, 15, 16,  7, 14, 15, 16, 12,
        17, 14, 18, 18, 14, 14, 17, 21, 12, 12,  9, 19, 12, 14, 12, 11, 10, 13,
        18, 14,  9, 11, 10, 12, 10, 25, 14, 18, 15, 16, 15, 18, 13,  9, 21, 11,
        20, 12, 13, 14, 14, 17, 10, 13, 18, 30, 14, 12, 13,  9, 10, 15, 13, 10,
        12, 15, 13, 18, 17, 13, 11, 12, 10, 16, 12, 13, 24, 14, 19, 19, 10, 20,
        12, 11], device='cuda:0')
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  

## Build Model
### Encoder

In [9]:
class Encoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lens):
        # src: (step, batch)
        embedded = self.dropout(self.emb(src))

        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_lens, enforce_sorted=False)
        # hidden/cell: (num_layers*num_directions, batch, hid_dim)
        packed_outs, (hidden, cell) = self.rnn(packed_embedded)
        return hidden, cell

In [10]:
SRC_IN_DIM = len(SRC.vocab)
TRG_IN_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
ENC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
DEC_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

encoder = Encoder(SRC_IN_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, ENC_PAD_IDX).to(device)
hidden, cell = encoder(batch_src, batch_src_lens)

print(batch_src.size())
print(hidden.size())
print(cell.size())

torch.Size([30, 128])
torch.Size([2, 128, 512])
torch.Size([2, 128, 512])


### Decoder

In [11]:
class Decoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        # The output dimension equals the input dimension for the decoder.
        self.fc = nn.Linear(hid_dim, in_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, cell):
        """
        One-step forward. 
        """
        # trg: (step=1, batch)
        embedded = self.dropout(self.emb(trg))

        # outs: (step=1, batch, hid_dim)
        # hidden/cell: (num_layers*num_directions, batch, hid_dim)
        outs, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # preds: (step=1, batch, out_dim=in_dim)
        preds = self.fc(outs)
        return preds, hidden, cell

In [12]:
decoder = Decoder(TRG_IN_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, DEC_PAD_IDX).to(device)
preds, hidden, cell = decoder(batch_trg[0].unsqueeze(0), hidden, cell)

print(batch_trg.size())
print(preds.size())
print(hidden.size())
print(cell.size())

torch.Size([29, 128])
torch.Size([1, 128, 5893])
torch.Size([2, 128, 512])
torch.Size([2, 128, 512])


### Seq2Seq

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lens, trg, teacher_forcing_ratio=0.5):
        # src: (step, batch)
        # trg: (step, batch)
        hidden, cell = self.encoder(src, src_lens)
        
        preds = []
        # The first input to the decoder is the <sos> token. 
        # trg_t: (step=1, batch)
        trg_t = trg[0].unsqueeze(0)
        for t in range(1, trg.size(0)):
            # preds_t: (step=1, batch, trg_out_dim)
            preds_t, hidden, cell = self.decoder(trg_t, hidden, cell)
            # top1: (step=1, batch)
            top1 = preds_t.argmax(dim=-1)
            if np.random.rand() < teacher_forcing_ratio:
                trg_t = trg[t].unsqueeze(0)
            else:
                trg_t = top1
            preds.append(preds_t)
        # preds: (step-1, batch, trg_out_dim)
        return torch.cat(preds, dim=0)

In [14]:
model = Seq2Seq(encoder, decoder).to(device)
preds = model(batch_src, batch_src_lens, batch_trg)

print(batch_trg.size())
print(preds.size())

torch.Size([29, 128])
torch.Size([28, 128, 5893])


## Train Model

In [15]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


encoder = Encoder(SRC_IN_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, ENC_PAD_IDX)
decoder = Decoder(TRG_IN_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, DEC_PAD_IDX)
model = Seq2Seq(encoder, decoder).to(device)

model.apply(init_weights)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


In [16]:
# Initialize Embeddings 
ENC_UNK_IDX = SRC.vocab.stoi[SRC.unk_token]
DEC_UNK_IDX = TRG.vocab.stoi[TRG.unk_token]

model.encoder.emb.weight.data[ENC_UNK_IDX].zero_()
model.encoder.emb.weight.data[ENC_PAD_IDX].zero_()
model.decoder.emb.weight.data[DEC_UNK_IDX].zero_()
model.decoder.emb.weight.data[DEC_PAD_IDX].zero_()

print(model.encoder.emb.weight[:5, :8])
print(model.decoder.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0171,  0.0020, -0.0057,  0.0012, -0.0194,  0.0050, -0.0010, -0.0051],
        [-0.0062, -0.0034, -0.0117, -0.0156, -0.0005, -0.0193, -0.0133, -0.0189],
        [ 0.0105, -0.0096,  0.0139,  0.0010,  0.0069, -0.0085, -0.0098, -0.0005]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0020,  0.0055, -0.0059, -0.0010, -0.0035, -0.0056,  0.0169, -0.0092],
        [-0.0002, -0.0016,  0.0042, -0.0044,  0.0022,  0.0113, -0.0221,  0.0069],
        [ 0.0065,  0.0097, -0.0082, -0.0009,  0.0024, -0.0109,  0.0009,  0.0053]],
       device='cuda:0', grad_fn=<SliceBackward>)


In [17]:
loss_func = nn.CrossEntropyLoss(ignore_index=DEC_PAD_IDX, reduction='mean')
optimizer = optim.AdamW(model.parameters())

We must ensure we turn `teacher forcing` off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment. 

In [18]:
def train_epoch(model, iterator, optimizer, loss_func, clip):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        # Forward pass
        batch_src, batch_src_lens = batch.src
        batch_trg, batch_trg_lens = batch.trg
        # preds: (step-1, batch, trg_out_dim)
        preds = model(batch_src, batch_src_lens, batch_trg)
        
        # Calculate loss
        preds_flattened = preds.view(-1, preds.size(-1))
        batch_trg_flattened = batch_trg[1:].flatten()
        loss = loss_func(preds_flattened, batch_trg_flattened)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update weights
        optimizer.step()
        # Accumulate loss
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

def eval_epoch(model, iterator, loss_func):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            batch_src, batch_src_lens = batch.src
            batch_trg, batch_trg_lens = batch.trg
            # preds: (step-1, batch, trg_out_dim)
            preds = model(batch_src, batch_src_lens, batch_trg, teacher_forcing_ratio=0)
            
            # Calculate loss
            preds_flattened = preds.view(-1, preds.size(-1))
            batch_trg_flattened = batch_trg[1:].flatten()
            loss = loss_func(preds_flattened, batch_trg_flattened)
            
            # Accumulate loss and acc
            epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [19]:
import time
N_EPOCHS = 10
CLIP = 1
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, loss_func, CLIP)
    valid_loss = eval_epoch(model, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 5.261 | Train PPL: 192.640
	 Val. Loss: 4.860 |  Val. PPL: 129.044
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 4.901 | Train PPL: 134.479
	 Val. Loss: 4.679 |  Val. PPL: 107.696
Epoch: 03 | Epoch Time: 0m 17s
	Train Loss: 4.415 | Train PPL:  82.722
	 Val. Loss: 4.459 |  Val. PPL:  86.378
Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 4.103 | Train PPL:  60.521
	 Val. Loss: 4.343 |  Val. PPL:  76.918
Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 3.909 | Train PPL:  49.829
	 Val. Loss: 4.180 |  Val. PPL:  65.387
Epoch: 06 | Epoch Time: 0m 17s
	Train Loss: 3.743 | Train PPL:  42.229
	 Val. Loss: 4.015 |  Val. PPL:  55.433
Epoch: 07 | Epoch Time: 0m 17s
	Train Loss: 3.579 | Train PPL:  35.855
	 Val. Loss: 3.956 |  Val. PPL:  52.243
Epoch: 08 | Epoch Time: 0m 17s
	Train Loss: 3.437 | Train PPL:  31.081
	 Val. Loss: 3.847 |  Val. PPL:  46.838
Epoch: 09 | Epoch Time: 0m 18s
	Train Loss: 3.311 | Train PPL:  27.408
	 Val. Loss: 3.786 |  Val. PPL:  44.099
E

In [20]:
model.load_state_dict(torch.load('models/tut1-model.pt'))

valid_loss = eval_epoch(model, valid_iterator, loss_func)
test_loss = eval_epoch(model, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')
print(f'Test Loss: {test_loss:.3f} |  Test PPL: {np.exp(test_loss):7.3f}')

Val. Loss: 3.704 |  Val. PPL:  40.617
Test Loss: 3.699 |  Test PPL:  40.412


## Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [21]:
print(model.encoder.emb.weight[:5, :8])
print(model.decoder.emb.weight[:5, :8])

tensor([[-0.0223,  0.0108,  0.0492,  0.0173,  0.0251, -0.0279,  0.0475, -0.0275],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0054, -0.0061,  0.0066,  0.0002, -0.0033, -0.0057,  0.0055, -0.0180],
        [-0.0386, -0.0408, -0.1120, -0.0399, -0.0249, -0.0212, -0.0749, -0.0723],
        [ 0.0138,  0.0117,  0.0195,  0.0115,  0.0369,  0.0285,  0.0062,  0.0149]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[ 0.0391, -0.0309, -0.0342,  0.0435,  0.0253, -0.0040, -0.0110,  0.0508],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0281, -0.0198, -0.0112,  0.0104,  0.0161,  0.0009, -0.0171,  0.0280],
        [-0.1392,  0.0918, -0.0215,  0.0861, -0.1324,  0.0398, -0.1677, -0.0257],
        [-0.0735,  0.0215, -0.0549,  0.0056, -0.0547, -0.0216, -0.0210, -0.1562]],
       device='cuda:0', grad_fn=<SliceBackward>)
